NLP on synopsis

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Data/2024_Processed.csv')
df.head(10)

,anime_id,title,type,source,episodes,rating,score,synopsis,producers,licensors,studios,genres,Popularity_category,Rank_category
0,1,Cowboy Bebop,TV,Original,26.0,R - 17+ (violence & profanity),8.75,"Crime is timeless. By the year 2071, humanity ...",['Bandai Visual'],['Funimation'],['Sunrise'],"['Action', 'Award Winning', 'Sci-Fi']",Top 100,Top 100
1,5,Cowboy Bebop: Tengoku no Tobira,Movie,Original,1.0,R - 17+ (violence & profanity),8.38,"Another day, another bounty—such is the life o...","['Sunrise', 'Bandai Visual']","['Sony Pictures Entertainment', 'Funimation']",['Bones'],"['Action', 'Sci-Fi']","Top 1,000",Top 500
2,6,Trigun,TV,Manga,26.0,PG-13 - Teens 13 or older,8.22,"Vash the Stampede is the man with a $$60,000,0...",['Victor Entertainment'],['Funimation'],['Madhouse'],"['Action', 'Adventure', 'Sci-Fi']",Top 500,Top 500
3,7,Witch Hunter Robin,TV,Original,26.0,PG-13 - Teens 13 or older,7.25,Robin Sena is a powerful craft user drafted in...,"['Bandai Visual', 'Dentsu', 'Victor Entertainm...","['Funimation', 'Bandai Entertainment']",['Sunrise'],"['Action', 'Drama', 'Mystery', 'Supernatural']","Top 5,000","Top 5,000"
4,8,Bouken Ou Beet,TV,Manga,52.0,PG - Children,6.99,It is the dark century and the people are suff...,"['TV Tokyo', 'Dentsu']",['Illumitoon Entertainment'],['Toei Animation'],"['Action', 'Adventure', 'Fantasy']","Top 7,500","Top 5,000"
5,15,Eyeshield 21,TV,Manga,145.0,PG-13 - Teens 13 or older,7.92,"Shy, reserved, and small-statured, Deimon High...","['TV Tokyo', 'Nihon Ad Systems', 'TV Tokyo Mus...","['VIZ Media', 'Sentai Filmworks']",['Gallop'],['Sports'],"Top 5,000","Top 1,000"
6,16,Hachimitsu to Clover,TV,Manga,24.0,PG-13 - Teens 13 or older,8.00,"Yuuta Takemoto, a sophomore at an arts college...","['Dentsu', 'Genco', 'Fuji TV', 'Asmik Ace', 'S...","['VIZ Media', 'Discotek Media']",['J.C.Staff'],"['Comedy', 'Drama', 'Romance']","Top 1,000","Top 1,000"
7,17,Hungry Heart: Wild Striker,TV,Manga,52.0,PG-13 - Teens 13 or older,7.57,As the younger brother of Japanese soccer star...,[],[],['Nippon Animation'],"['Comedy', 'Slice of Life', 'Sports']","Top 5,000","Top 5,000"
8,18,Initial D Fourth Stage,TV,Manga,24.0,PG-13 - Teens 13 or older,8.17,Takumi Fujiwara finally joins Ryousuke and Kei...,"['OB Planning', 'Studio Jack']",['Funimation'],['A.C.G.T.'],"['Action', 'Drama']","Top 5,000",Top 500
9,19,Monster,TV,Manga,74.0,R+ - Mild Nudity,8.88,"Dr. Kenzou Tenma, an elite neurosurgeon recent...","['VAP', 'Shogakukan-Shueisha Productions', 'Ni...",['VIZ Media'],['Madhouse'],"['Drama', 'Mystery', 'Suspense']",Top 500,Top 100


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216 entries, 0 to 20215
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   anime_id             20216 non-null  int64  
 1   title                20216 non-null  object 
 2   type                 20216 non-null  object 
 3   source               20216 non-null  object 
 4   episodes             20216 non-null  float64
 5   rating               20216 non-null  object 
 6   score                20216 non-null  float64
 7   synopsis             17175 non-null  object 
 8   producers            20216 non-null  object 
 9   licensors            20216 non-null  object 
 10  studios              20216 non-null  object 
 11  genres               20216 non-null  object 
 12  Popularity_category  20216 non-null  object 
 13  Rank_category        20216 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 2.2+ MB


In [4]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk
from nltk.tree import Tree

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-

True

In [5]:
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

custom_words = {'and', 'the', 'is', 'a', 'to', 'it', 's', 'like', 'year'}
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in custom_words) + r')\b'

def is_capitalized(word):
    return word[0].isupper() and word.isalpha()

# Too computationally expensive, not used
def is_name(word, pos_tagged):
    for chunk in ne_chunk(pos_tagged):
        if isinstance(chunk, Tree):
            for leaf in chunk.leaves():
                if leaf[0] == word:
                    return True
    return False

def lemmatize_text(text):
    if pd.isnull(text): 
        return text
    words = word_tokenize(text)
    pos_tagged = pos_tag(words)

    lemmatized_words = [
        lemmatizer.lemmatize(word.lower()) for word in words
        if word.lower() not in stop_words and word.lower() not in custom_words and not is_capitalized(word)
    ]
    lemmatized_text = ' '.join(lemmatized_words)

    ## Futher clean anything lemmatization missed, remove spaces and characters
    cleaned_text = re.sub(r'[^\w\s]', '', lemmatized_text)
    cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

In [6]:
# Test NLTK

nlp_test = "Crime is timeless. By the year 2071, humanity has expanded across the galaxy, filling the surface of other planets with settlements like those on Earth. These new societies are plagued by murder, drug use, and theft, and intergalactic outlaws are hunted by a growing number of tough bounty hunters. Spike Spiegel and Jet Black pursue criminals throughout space to make a humble living. Beneath his goofy and aloof demeanor, Spike is haunted by the weight of his violent past. Meanwhile, Jet manages his own troubled memories while taking care of Spike and the Bebop, their ship. The duo is joined by the beautiful con artist Faye Valentine, odd child Edward Wong Hau Pepelu Tivrusky IV, and Ein, a bioengineered Welsh Corgi. While developing bonds and working to catch a colorful cast of criminals, the Bebop crew's lives are disrupted by a menace from Spike's past. As a rival's maniacal plot continues to unravel, Spike must choose between life with his newfound family or revenge for his old wounds."
nlp_test = lemmatize_text(nlp_test)
nlp_test

'timeless 2071 humanity expanded across galaxy filling surface planet settlement new society plagued murder drug use theft intergalactic outlaw hunted growing number tough bounty hunter pursue criminal throughout space make humble living goofy aloof demeanor haunted weight violent past manages troubled memory taking care ship duo joined beautiful con artist odd child bioengineered developing bond working catch colorful cast criminal crew life disrupted menace past rival maniacal plot continues unravel must choose life newfound family revenge old wound'

In [7]:
df['synopsis'] = df['synopsis'].apply(lemmatize_text)

In [8]:
df['synopsis'].head(10)

0    timeless 2071 humanity expanded across galaxy ...
1    day another bountysuch life often unlucky crew...
2    man 60000000000 bounty head reason merciless v...
3    powerful craft user drafted stnja group specia...
4    dark century people suffering rule devil able ...
5    reserved smallstatured student perfect target ...
6    sophomore art college share cheap apartment tw...
7    younger brother soccer star always expected gr...
8    finally join create d goal twofold want develo...
9    dr elite neurosurgeon recently engaged hospita...
Name: synopsis, dtype: object

In [9]:
df.to_csv('nlp.csv', index = False)